In [1]:
import pennylane as qml
import pennylane.numpy as np

$H = - \sum^{N}_{i=1}Z_i \otimes Z_{i+1} - h\sum^{N}_{i}X_i$

In [10]:
num_qubits = 4
def create_Hamiltonian(h):
    coeffs1 =  num_qubits*[-1]
    obs1 = [qml.PauliZ(wire)@ qml.PauliZ((wire+1)% num_qubits) for wire in range(num_qubits)]

    coeffs2 = num_qubits*[-h]
    obs2 = [qml.PauliX(wire) for wire in range(num_qubits)]

    coeffs = coeffs1 + coeffs2
    obs = obs1 + obs2

    return qml.Hamiltonian(coeffs,obs)

In [4]:
dev = qml.device("default.qubit",wires=num_qubits)

@qml.qnode(dev)
def model(params, H):
    qml.BasicEntanglerLayers(weights=params[0],wires=range(4),rotation=qml.RX)
    qml.BasicEntanglerLayers(weights=params[1],wires=range(4),rotation=qml.RY)
    qml.BasicEntanglerLayers(weights=params[2],wires=range(4),rotation=qml.RZ)

    return  qml.expval(H)

In [8]:
def train(h):
    opt = qml.GradientDescentOptimizer(stepsize=0.1)
    max_iterations = 1000
    conv_tol = 1e-4

    Hamiltonian = create_Hamiltonian(h)
    params = np.random.rand(3,3,4, requires_grad=True)

    for n in range(max_iterations):
        params, prev_cost = opt.step_and_cost(model,params,H = Hamiltonian)
        new_cost = model(params, Hamiltonian)

        if n % 20 == 0:
            print(f"Step = {n}, Cost function = {new_cost: 0f}")
        if np.abs(new_cost - prev_cost) <= conv_tol:
            break
    return params

In [11]:
inp = 1.0

param = train(inp)

Step = 0, Cost function = -1.347378
Step = 20, Cost function = -4.001592
Step = 40, Cost function = -4.655360
Step = 60, Cost function = -4.870438
Step = 80, Cost function = -4.910694
Step = 100, Cost function = -4.940990
Step = 120, Cost function = -4.973631
Step = 140, Cost function = -5.013870
Step = 160, Cost function = -5.066217
Step = 180, Cost function = -5.125195
Step = 200, Cost function = -5.172557
Step = 220, Cost function = -5.200302
Step = 240, Cost function = -5.213252
Step = 260, Cost function = -5.218681


## Ising Uprising

**Backstory**
Zenda and Reece model Sqynet as a spin chain, and they come up with a strategy. What if, in addition to using plasma bombs and missiles to increase the temperature of the device, they use a strong magnetic field? After all, magnetic fields might pass through Sqynet's outer shell more easily. The scientists proceed to simulate the effect of a magnetic field on a closed spin chain to quantify the effects.

**Ground state of an Ising spin chain**
A simple way to model Sqynet is by considering it as a closed spin chain of length $N$. A spin chain contains particles of spin $1/2$ in each of its $N$ sites. The spins may be pointing in the positive or negative $z$ direction, and we consider that there may be an external magnetic field acting on the system.
![system](./images/Ising%20Uprising_1.png)
Such a quantum system is described by the Transverse Ising Hamiltonian. For closed spin chain with a transverse magnetic field of intensity , the Transverse Ising Hamiltonian reads
$$H = - \sum_{i=1}^{N}(Z_i\otimes Z_{i+1} - h\sum_{i=1}^{N}X_i.$$
The subindices $i$ indicate the spin site where the operators act. In a closed spin chain, we identify site $N+1$ with the first site.

A possible plan for Zenda and Reece is to use a strong magnetic field that changes the ground energy of Sqynet, causing it to malfunction.
Your task is to help Zenda and Reece calculate the effect of external magnetic forces on the ground energy. Using the Variational Quantum Eigensolver (VQE) algorithm, you will compute the ground energy of a closed spin chain of length $N=4$

**Challenge code**

In this challenge you will be given the following functions:

* create_Hamiltonian: In which you build the Transverse Ising Hamiltonian for $N=4$ and a magnetic field intensity h. You must complete this function.
* model: This QNode builds a general enough ansatz for the ground state. This circuit must depend on some parameters params, which you will later optimize. It returns the expectation value of the Hamiltonian for the output state of the circuit. You must complete this function.
* train: This function returns the parameters that minimize the output of model. You must complete this function.

**Input**
As input to this problem, you are given:

* h (float): Magnetic field intensity applied to the spin chain.

**Output**
This code will output a float corresponding to the energy of the ground state.


In [13]:
import json
import pennylane as qml
import pennylane.numpy as np

def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """
    coeffs1 =  num_qubits*[-1]
    obs1 = [qml.PauliZ(wire)@ qml.PauliZ((wire+1)% num_qubits) for wire in range(num_qubits)]

    coeffs2 = num_qubits*[-h]
    obs2 = [qml.PauliX(wire) for wire in range(num_qubits)]

    coeffs = coeffs1 + coeffs2
    obs = obs1 + obs2

    return qml.Hamiltonian(coeffs,obs)


dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """
    qml.BasicEntanglerLayers(weights=params[0],wires=range(4),rotation=qml.RX)
    qml.BasicEntanglerLayers(weights=params[1],wires=range(4),rotation=qml.RY)
    qml.BasicEntanglerLayers(weights=params[2],wires=range(4),rotation=qml.RZ)

    return  qml.expval(H)


def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """
    opt = qml.GradientDescentOptimizer(stepsize=0.1)
    max_iterations = 1000
    conv_tol = 1e-4

    Hamiltonian = create_Hamiltonian(h)
    params = np.random.rand(3,3,4, requires_grad=True)

    for n in range(max_iterations):
        params, prev_cost = opt.step_and_cost(model,params,H = Hamiltonian)
        new_cost = model(params, Hamiltonian)

        if np.abs(new_cost - prev_cost) <= conv_tol:
            break
    return params


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."


test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.0'...
Correct!
Running test case 1 with input '2.3'...
Correct!
